In [1]:
#!/usr/bin/env python3
import os, sys, signal,rospy, argparse, csv

from tqdm import tqdm
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.spatial.distance import cdist
import copy, pickle
import open3d as o3d
import open3d.visualization as vis
import torch
from torch.autograd import Variable

from open3d_ros_helper import open3d_ros_helper as orh
from geometry_msgs.msg import Pose, PoseArray, Point # PoseArray, Pose
from std_msgs.msg import ColorRGBA
from sensor_msgs.msg import PointCloud2
from visualization_msgs.msg import Marker

from models.mdgat import MDGAT

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
parser = argparse.ArgumentParser(
    description='Point cloud matching and pose evaluation',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter)

opt = argparse.Namespace(
    slam_dir = '/media/vision/Seagate/DataSets/KRGM/kitti/',
    data_folder = 'harris_3D',
    local_global = False,
    seq_num = '00',
    visualize = False,
    vis_line_width = 0.2,
    calculate_pose = True,
    learning_rate = 0.0001,
    batch_size = 1,
    train_path = './KITTI/',
    model_out_path = './models/checkpoint',
    memory_is_enough = True,
    local_rank = 0,
    txt_path = './KITTI/preprocess-random-full',
    keypoints_path = './KITTI/keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise',
    resume_model = './checkpoint/kitti/mdgat-l9-gap_loss-pointnetmsg-04_01_19_32/train_step3/nomutualcheck-mdgat-batch16-gap_loss-pointnetmsg-USIP-04_01_19_32/best_model_epoch_221(val_loss0.31414552026539594).pth',
    loss_method = 'triplet_loss',
    net = 'mdgat',
    mutual_check = False,
    k = [128, None, 128, None, 64, None, 64, None],
    l = 9,
    descriptor = 'pointnetmsg',
    keypoints = 'USIP',
    ensure_kpts_num = False,
    max_keypoints = -1,
    match_threshold = 0.2,
    threshold = 0.5,
    triplet_loss_gamma = 0.5,
    sinkhorn_iterations = 20,
    train_step = 3,

    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.0,
    seq_list = [0],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    transform_opt = 0
)

In [3]:
def handle_sigint(signal, frame):
    print("\n ---cancel by user---")
    sys.exit(0)

def model_inference(net, data, device):
    net.eval()
    with torch.no_grad():
        data['keypoints0'] = data['keypoints0'].to(device)
        data['keypoints1'] = data['keypoints1'].to(device)
        data['cloud0'] = data['cloud0'].to(device)
        data['cloud1'] = data['cloud1'].to(device)
        data['scores0'] = data['cloud0'].to(device)
        data['scores1'] = data['cloud1'].to(device)

        output = net(data)
        return output

In [4]:
class dataset():
    def __init__(self, args) -> None:
        self.gt_seq = args.seq_num
        self.data_folder = args.data_folder

        self.dir_SLAM_path = args.slam_dir + self.data_folder +"/"
        self.descriptor_type = args.descriptor

        self.poses = []
        self.dense_scans = []
        self.keypoints = []
        self.descriptors = []
        self.local_graph_range = [0, 0]
        self.divided_keypoints = np.array([])
        self.divided_dense_scans = []

        self._get_SLAM_poses()
        self._get_dense_frames()
        self._get_keypoints()
        self._get_descriptors()
        print("[Load] %d's poses SLAM data loaded" % len(self.poses))

    def _get_SLAM_poses(self):
        with open(file=os.path.join(self.dir_SLAM_path, "Poses_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
            self.poses = pickle.load(f)

    def _get_dense_frames(self):
        with open(file=os.path.join(self.dir_SLAM_path, "DenseFrames_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
            self.dense_scans = pickle.load(f)

    def _get_keypoints(self):
        with open(file=os.path.join(self.dir_SLAM_path, "keyPoints_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
            self.keypoints = pickle.load(f) # (pose_num, keypoint_num, 3), (keypoint_num, 3)는 np.array

    def _get_descriptors(self):
        if self.descriptor_type == "FPFH":
            with open(file=os.path.join(self.dir_SLAM_path, "Descriptors_FPFH_kitti_" + self.gt_seq + ".pickle"), mode='rb') as f:
                self.descriptors = pickle.load(f)
        elif self.descriptor_type == "pointnet" or "pointnetmsg":
            for kps in self.keypoints:
                self.descriptors.append(torch.zeros((kps.shape[0], 128)))

    def divide_data(self, divide_num = 256):
        # divide keypoints
        flatten_keypoints = []
        for kps in self.keypoints:
            for kp in kps:
                flatten_keypoints.append(np.array(kp))
        flatten_keypoints = np.array(flatten_keypoints) # (n, 3)
        sample_num = flatten_keypoints.shape[0] // (divide_num)
        flatten_keypoints = flatten_keypoints[:(divide_num) * sample_num]
        self.divided_keypoints = flatten_keypoints.reshape((sample_num, divide_num, 3))

        # devide dense_scans
        n = 0
        divided_pcd = o3d.geometry.PointCloud()
        for idx, kps in enumerate(self.keypoints):
            divided_pcd.points.extend(self.dense_scans[idx])
            for _ in kps:
                n += 1
                if n % divide_num == 0:
                    divided_pcd = divided_pcd.voxel_down_sample(voxel_size=0.2)
                    self.divided_dense_scans.append(np.array(divided_pcd.points))
                    divided_pcd.clear()
                    divided_pcd.points.extend(self.dense_scans[idx])
                    n = 0
        print("[Divide] Keypoints and dense scan divided into %d-num seq" % (sample_num))

    def get_divided_data(self, idx):
        if idx >= len(self.divided_keypoints) - 1:
            idx = len(self.divided_keypoints) - 1
            print("[Warning] idx is over the divided keypoints, so return the last one")
        infer_kp = torch.tensor(self.divided_keypoints[idx], dtype=torch.double).unsqueeze(0)
        infer_pc = torch.tensor(self.divided_dense_scans[idx], dtype=torch.double)
        infer_pc = torch.cat((infer_pc, torch.ones(infer_pc.shape[0], 1)), dim=1)
        
        n = infer_pc.shape[0] // 2
        infer_pc = infer_pc[:n*2,:]
        infer_pc = infer_pc.reshape((-1, 8)).unsqueeze(0)
        
        infer_scores = torch.ones((infer_kp.shape[1])).unsqueeze(0)

        infer_kp.to('cuda')
        infer_pc.to('cuda')
        infer_scores.to('cuda')

        return{
            'keypoints0': infer_kp,
            'cloud0': infer_pc,
            'scores0': infer_scores
        } 
    
    def push_descriptor(self, descriptors):
        for desc in descriptors:
            self.descriptors.append(desc)

    def get_matching_data(self, idx):
        if idx >= len(self.divided_keypoints) - 1:
            idx = len(self.divided_keypoints) - 1
            print("[Warning] idx is over the divided keypoints, so return the last one")

        global_keypoints = np.zeros((1,3))
        global_descriptors = np.zeros((1,256))

        for i in range(idx - 1):
            global_keypoints = np.vstack((global_keypoints, self.divided_keypoints[i]))
            global_descriptors = np.vstack((global_descriptors, self.divided_dense_scans[i]))
        local_keypoints = self.divided_keypoints[idx]
        local_descriptors = self.divided_dense_scans[idx]
                                     

    def __len__(self):
        return len(self.poses)

dataloader 테스트

In [ ]:
dataset = dataset(opt)
dataset.divide_data(256)

In [ ]:
for idx in range(len(dataset.divided_dense_scans)+5):
    kp, pc = dataset.get_divided_data(idx)
    print(idx,"-th data:", kp.shape, pc.shape)

In [13]:
for idx in range(len(dataset.divided_dense_scans)):
    print( idx, "-th section: ", dataset.divided_dense_scans[idx].shape, ", ", dataset.divided_keypoins[idx].shape)
    pcd_pc = o3d.geometry.PointCloud()
    pcd_pc.points = o3d.utility.Vector3dVector(dataset.divided_dense_scans[idx])
    pcd_pc.colors = o3d.utility.Vector3dVector(np.ones((dataset.divided_dense_scans[idx].shape[0], 3)) * [0.5, 0.5, 0.5])
    pcd_kp = o3d.geometry.PointCloud()
    pcd_kp.points = o3d.utility.Vector3dVector(dataset.divided_keypoins[idx])
    pcd_kp.colors = o3d.utility.Vector3dVector(np.ones((dataset.divided_keypoins[idx].shape[0], 3)) * [1, 0, 0])

    # Visualize the point clouds
    # vis.draw_geometries([pcd_pc, pcd_kp])

AttributeError: 'dataset' object has no attribute 'divided_keypoins'

In [ ]:
for idx in range(len(dataset.divided_keypoints)):
    kp, desc = dataset.get_matching_data(idx)
    print()

emplimentation 테스트 시나리오

In [5]:
# 1. 데이터 로드 및 분할
dataset = dataset(opt)
dataset.divide_data(256)

[Load] 929's poses SLAM data loaded
[Divide] Keypoints and dense scan divided into 42-num seq


In [6]:
pred = dataset.get_divided_data(2)
print(pred['keypoints0'].shape, pred['cloud0'].shape, pred['scores0'].shape)    

torch.Size([1, 256, 3]) torch.Size([1, 54621, 8]) torch.Size([1, 256])


In [7]:
# 2. 모델 로드
path_checkpoint = opt.resume_model  
checkpoint = torch.load(path_checkpoint, map_location={'cuda:2':'cuda:0'})  
lr = checkpoint['lr_schedule']
config = {
    'net': {
        'sinkhorn_iterations': opt.sinkhorn_iterations,
        'match_threshold': opt.match_threshold,
        'lr': opt.learning_rate,
        'loss_method': opt.loss_method,
        'k': opt.k,
        'descriptor': opt.descriptor,
        'mutual_check': opt.mutual_check,
        'triplet_loss_gamma': opt.triplet_loss_gamma,
        'train_step':opt.train_step,
        'L':opt.l
    }
}
net = MDGAT(config.get('net', {}))
optimizer = torch.optim.Adam(net.parameters(), lr=config.get('net', {}).get('lr'))
net = torch.nn.DataParallel(net)
net.load_state_dict(checkpoint['net'])

if torch.cuda.is_available():
    # torch.cuda.set_device(opt.local_rank)
    device=torch.device('cuda:{}'.format(opt.local_rank))
else:
    device = torch.device("cpu")
    print("### CUDA not available ###")

net.to(device)

DataParallel(
  (module): MDGAT(
    (penc): PointnetEncoderMsg(
      (sa1): PointNetSetKptsMsg(
        (conv_blocks): ModuleList(
          (0): ModuleList(
            (0): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
          )
          (1): ModuleList(
            (0): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): ModuleList(
            (0): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1))
            (1): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1))
            (2): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (bn_blocks): ModuleList(
          (0): ModuleList(
            (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_r

In [11]:
# 3. 디스크립터 추출
net.double().eval()
pred = dataset.get_divided_data(2)

for p in pred:
    pred[p] = pred[p].to(device)

data = net.module.infer(pred)
pred = {**pred, **data}


In [12]:
print(data.keys())

dict_keys(['desc'])
